In [1]:
var h = 0.00001
var a  = 3
var b = 4
var c = 5 + h 

var d = (a*b + c - (a*b + c - h) )/(h)

In [2]:
class Value {
  constructor(data, _children = [], _op = '', _label= '') {
    this.data = data;           // The actual value
    this._children = new Set(_children); // Set of child nodes in the computational graph
    this._backward = () => {};  // Function to propagate gradients
    this._op = _op;             // Operation used to create this value (add, mul, tanh, etc.)
    this.grad = 0;              // Gradient initialized to 0
    this._label = _label
  }

  // Addition operation
  add(operand) {
    if( !(operand instanceof Value)) {
        operand = new Value(operand)
    }
    // Create the output Value from the addition of two inputs
    const out = new Value(this.data + operand.data, [this, operand], '+');
    
    // Define the backward function for this addition
    out._backward = () => {
      this.grad += out.grad;    // Gradient for addition is passed equally to both operands
      operand.grad += out.grad;
    };
    
    return out;  // Return the result Value
  }

  // Multiplication operation
  mul(operand) {
    if( !(operand instanceof Value)) {
        operand = new Value(operand)
    }
    // Create the output Value from the multiplication of two inputs
    const out = new Value(this.data * operand.data, [this, operand], '*');
    
    // Define the backward function for this multiplication
    out._backward = () => {
      this.grad += operand.data * out.grad; // Gradient for multiplication w.r.t this
      operand.grad += this.data * out.grad; // Gradient for multiplication w.r.t operand
    };
    
    return out;  // Return the result Value
  }

  // Tanh activation function
    tanh() {
        const tanhData = Math.tanh(this.data);  // Compute tanh of the current value
        const out = new Value(tanhData, [this], 'tanh');

        // Define the backward function for tanh
        out._backward = () => {
          const tanhGrad = 1 - tanhData ** 2;  // Derivative of tanh(x) is 1 - tanh(x)^2
          this.grad += tanhGrad * out.grad;    // Chain rule: multiply by the gradient of the output
        };

        return out;  // Return the result Value
    }
    
    exp() {
        // Compute exponentiation
        const exp = Math.exp(this.data)
        const out = new Value(exp, [this], 'exp');

        // Define the backward function for this exponentiation
        out._backward = () => {
          this.grad += exp * out.grad; // Gradient w.r.t base
        };

        return out;  // Return the result Value
      }
    
    pow(operand) {
        // Compute exponentiation
        const out = new Value(Math.pow(this.data, operand), [this], '^');

        // Define the backward function for this exponentiation
        out._backward = () => {
          this.grad += operand * Math.pow(this.data, operand -1)*out.grad; // Gradient w.r.t base
        };

        return out;  // Return the result Value
    }
    
    sub(operand) {
        if( !(operand instanceof Value)) {
            operand = new Value(operand)
        }
        return this.add(operand.mul(-1))
    }
    
    div(operand) {
        return this.mul(operand.pow(-1))
    }
    
    backward() {
      // Topologically sort the nodes in the computational graph
      const topo = [];
      const visited = new Set();

      // Helper function to perform depth-first search (DFS) to order nodes
      const buildTopo = (v) => {
        if (!visited.has(v)) {
          visited.add(v);
          for (let child of v._children) {
            buildTopo(child);
          }
          topo.push(v);
        }
      };

      // Build the topological order starting from this node
      buildTopo(this);

      // Initialize the gradient of the output node (this node) to 1
      this.grad = 1;

      // Backpropagate through all nodes in reverse topological order
      for (let node of topo.reverse()) {
        node._backward();
      }
    }
}
    

In [3]:
class Neuron{
    constructor(num_in) {
        this.weights = Array.from({ length: num_in }, () => new Value(Math.random() * 2 - 1));
        this.bias = new Value(Math.random() * 2 - 1)
    }
    
    parameters(){
        let params_list = this.weights.concat([this.bias]);
        return params_list;
    }

    zero_grad() {
        this.weights.forEach(weight => {
            weight.grad = 0
        });

        this.bias.grad = 0
    }
    
    forward(x) {
        if (x.length !== this.weights.length) {
            throw new Error('Input vector length must match the number of neuron weights.');
        }
        let result = this.weights.reduce((sum, weight, i) => sum.add(weight.mul(x[i])), this.bias);
        return result.tanh();
    }
}

class Layer {
    constructor(num_in, num_o) {
        this.num_in = num_in
        this.neurons = Array.from({ length: num_o }, () => new Neuron(num_in));
    }
    
    parameters() {
        return this.neurons.flatMap(neuron => neuron.parameters());
    }
    
    zero_grad() {
        this.neurons.forEach(neuron => {
            neuron.zero_grad()
        });
    }

    forward(x) {
        const outs = this.neurons.map(neuron => neuron.forward(x));
        return outs
    }
}

class MLP {
    constructor(num_in, layer_sizes) {
        this.layers = [];
        this.layers.push(new Layer(num_in, layer_sizes[0]));

        for (let i = 1; i < layer_sizes.length; i++) {
            this.layers.push(new Layer(layer_sizes[i - 1], layer_sizes[i]));
        }
    }
    
    parameters() {
        return this.layers.flatMap(layer => layer.parameters());
    }

    zero_grad()
    {
        this.layers.forEach(layer => {
            layer.zero_grad();
        });
    }
    
    forward(x) {
        let out = x;
        for (let layer of this.layers) {
            out = layer.forward(out);
        }
        if(out.length == 1) {
            return out[0]
        }
        return out;
    }
}

In [4]:
var x1 = new Value(2.0, [], '', 'x1');
var x2 = new Value(0.0, [], '', 'x2');

var w1 = new Value(-3, [], '', 'w1');
var w2 = new Value(1.0, [], '', 'w2');

var b = new Value(6.8813735870195432, [], '', 'b');

var x1w1 = x1.mul(w1)
x1w1._label = 'x1w1'
var x2w2 = x2.mul(w2)
x1w1._label = 'x2w2'

var x1w1x2w2 = x1w1.add(x2w2)
x1w1x2w2._label='x1w1x2w2'
var n = x1w1x2w2.add(b)
n._label = 'n'
// var o = n.tanh()
// o.backward()

"n"

In [5]:
var num = n.mul(2).exp().sub(1)
var denom = n.mul(2).exp().add(1)
var o = num.div(denom)
o.data

0.7071067811865477

In [6]:
o.backward()

In [7]:
var mlp = new MLP(3,  [4, 3, 2])

In [8]:
mlp.forward([1, 2, 3])

[
  Value {
    data: -0.7618572456693513,
    _children: Set(1) {
      Value {
        data: -1.000626741447334,
        _children: Set(2) { [Value], [Value] },
        _backward: [Function (anonymous)],
        _op: "+",
        grad: 0,
        _label: ""
      }
    },
    _backward: [Function (anonymous)],
    _op: "tanh",
    grad: 0,
    _label: ""
  },
  Value {
    data: -0.910187241242226,
    _children: Set(1) {
      Value {
        data: -1.5286147516880508,
        _children: Set(2) { [Value], [Value] },
        _backward: [Function (anonymous)],
        _op: "+",
        grad: 0,
        _label: ""
      }
    },
    _backward: [Function (anonymous)],
    _op: "tanh",
    grad: 0,
    _label: ""
  }
]

# Generating Dataset

In [9]:
function targetFunction(x1, x2, x3) {
    return Math.sin(x1) + Math.pow(x2, 2) - Math.log(1 + Math.abs(x3));
}

// Function to generate synthetic data using the target function
function generateLearnableDataset(num_samples, num_features) {
    const data = [];
    const labels = [];

    for (let i = 0; i < num_samples; i++) {
        // Generate random input vector of size num_features
        let input = Array.from({ length: num_features }, () => Math.random() * 2 - 1);

        // Apply the target function to generate the label
        let labelValue = targetFunction(input[0], input[1], input[2]);
        let label = new Value(labelValue);  // Wrapping in the Value class

        // Convert the input to Value objects for the neural network
        let inputValues = input.map(val => new Value(val));

        data.push(inputValues);
        labels.push(label);
    }

    return { data, labels };
}
const { data, labels } = generateLearnableDataset(30, 3)

In [14]:
var mlp = new MLP(3,  [5, 6, 1])

In [15]:
for(let i =0; i < 1000; i++) {
    let outputs = Array.from({ length: data.length }, (_, i) => (mlp.forward(data[i])));
    let loss = outputs.reduce((sum, output, i) => sum.add(output.sub(labels[i]).pow(2)), new Value(0));
    
    
    mlp.zero_grad()
    loss.backward()
    var lr = 0.005
    var parameters = mlp.parameters()
    parameters.forEach(parameter => {
        parameter.data += -lr * parameter.grad;
    });
    if (i %20 == 0) {
        console.log(i, loss.data)
    }
}



0 31.464631213859054
20 5.449050247648166
40 3.4496698995989297
60 2.7169446760252822
80 2.33418475156317
100 2.103479009807744
120 1.9102392203859142
140 1.708749632650104
160 1.4868049203143543
180 1.2512868139854794
200 1.0236222047835573
220 0.8259344767213348
240 0.6683185206077529
260 0.5505722366306237
280 0.467794241374118
300 0.4124648418596669
320 0.37626325039124536
340 0.3521710050042297
360 0.33536289024995225
380 0.32292993781220003
400 0.31323092478227327
420 0.30534806032576894
440 0.29874753182741315
460 0.29309642731309327
480 0.2881708886909051
500 0.28381096242346043
520 0.27989777560021467
540 0.2763411088034541
560 0.27307187675512085
580 0.2700370671627997
600 0.26719604771003697
620 0.26451773982673643
640 0.26197840846755466
660 0.2595599240144589
680 0.25724839981457626
700 0.2550331322414585
720 0.2529057840340665
740 0.25085976170950786
760 0.24888974614071377
780 0.2469913426100162
800 0.2451608229675912
820 0.24339493797525527
840 0.24169078252646914
860 0